In [121]:
import numpy as np
import pandas as pd
from datetime import datetime
df = pd.read_excel('EyeTrackerData.xlsx')

In [122]:
df = df.loc[:,["Sensor Time", "Lft X Pos", "Lft Y Pos", "Lft Pupil Diameter","Rt X Pos", "Rt Y Pos","Rt Pupil Diameter","Capture Time"]]

In [123]:
df['gaze_x'] = (df['Lft X Pos'] + df["Rt X Pos"])/2
df['gaze_y'] = (df['Lft Y Pos'] + df["Rt Y Pos"])/2

In [124]:
df['time_convert'] = df['Capture Time'].apply(lambda x: datetime.utcfromtimestamp(x/1000).strftime('%Y-%m-%d %H:%M:%S'))

In [125]:
df['time_convert'] = df['time_convert'].apply(lambda x: datetime.strptime(x,"%Y-%m-%d %H:%M:%S"))

In [126]:
df

,Sensor Time,Lft X Pos,Lft Y Pos,Lft Pupil Diameter,Rt X Pos,Rt Y Pos,Rt Pupil Diameter,Capture Time,gaze_x,gaze_y,time_convert
0,71789417767,0.0000,0.0000,0.000000,0.0000,0.0000,0.000000,71789578202,0.0000,0.0000,1972-04-10 21:32:58
1,71789433958,0.0000,0.0000,0.000000,0.0000,0.0000,0.000000,71789595159,0.0000,0.0000,1972-04-10 21:33:15
2,71789451202,0.0000,0.0000,0.000000,0.0000,0.0000,0.000000,71789622078,0.0000,0.0000,1972-04-10 21:33:42
3,71789466661,0.0000,0.0000,0.000000,0.0000,0.0000,0.000000,71789625068,0.0000,0.0000,1972-04-10 21:33:45
4,71789482596,0.0000,0.0000,0.000000,0.0000,0.0000,0.000000,71789646185,0.0000,0.0000,1972-04-10 21:34:06
...,...,...,...,...,...,...,...,...,...,...,...
13216,72003841288,1007.5436,406.4036,3.268553,1007.5436,406.4036,3.050074,72004012114,1007.5436,406.4036,1972-04-13 09:06:52
13217,72003858295,1012.7636,405.9277,3.370696,1012.7636,405.9277,3.108841,72004024075,1012.7636,405.9277,1972-04-13 09:07:04
13218,72003873570,1018.0250,405.0260,3.343998,1018.0250,405.0260,3.162790,72004036040,1018.0250,405.0260,1972-04-13 09:07:16
13219,72003889708,1023.9369,403.5870,3.367016,1023.9369,403.5870,3.184986,72004055981,1023.9369,403.5870,1972-04-13 09:07:35


In [127]:
def blink_detection(x, y, time, missing=0.0, minlen=10):

	"""Detects blinks, defined as a period of missing data that lasts for at
	least a minimal amount of samples

	arguments
	x		-	numpy array of x positions
	y		-	numpy array of y positions
	time		-	numpy array of EyeTribe timestamps
	keyword arguments
	missing	-	value to be used for missing data (default = 0.0)
	minlen	-	integer indicating the minimal amount of consecutive
				missing samples

	returns
	Sblk, Eblk
				Sblk	-	list of lists, each containing [starttime]
				Eblk	-	list of lists, each containing [starttime, endtime, duration]
	"""

	# empty list to contain data
	Sblk = []
	Eblk = []

	# check where the missing samples are
	mx = np.array(x==missing, dtype=int)
	my = np.array(y==missing, dtype=int)
	miss = np.array((mx+my) == 2, dtype=int)

	# check where the starts and ends are (+1 to counteract shift to left)
	diff = np.diff(miss)
	starts = np.where(diff==1)[0] + 1
	ends = np.where(diff==-1)[0] + 1

	# compile blink starts and ends
	for i in range(len(starts)):
		# get starting index
		s = starts[i]
		# get ending index
		if i < len(ends):
			e = ends[i]
		elif len(ends) > 0:
			e = ends[-1]
		else:
			e = -1
		# append only if the duration in samples is equal to or greater than
		# the minimal duration
		if s-e >= minlen:
			# add starting time
			Sblk.append([time[s]])
			# add ending time
			Eblk.append([time[s],time[e],(time[e]-time[s]).seconds/1000])

	return Sblk,Eblk

In [128]:
Sblk, Eblk = blink_detection(df['gaze_x'], df['gaze_y'], df['time_convert'], minlen=10)

## Calculating the blink rate
### To calculate the blink rate we have the the function below

In [129]:
from datetime import datetime
from pandas import Timestamp


def calculate_blink_rate(list1):
    #the line below will traverse through the second element of the sublist in the list Eblk
    #Eblk = list of lists, each containing [starttime, endtime, duration]. Therefore we need the value duration
    #we will then calulate the sum which will give us the total blink time
    total_duration_seconds = sum(entry[2] for entry in Eblk)
    #total_blinks will give us the number of blinks in the data
    total_blinks = len(Eblk)
    #the blink rate is calculated by totalblinks divided by the total time taken to blink
    blink_rate = total_blinks / total_duration_seconds
    print("Blink rate is", blink_rate,"per unit time")
    
calculate_blink_rate(Eblk)

Blink rate is 0.011860113704158515 per unit time


## Now for below we will be working on the fixation 

In [199]:
data = pd.read_excel('EyeTrackerData.xlsx')

In [200]:
data

,Obsv.,Frame,Capture Time,System Time,Sensor Time,Lft X Pos,Lft Y Pos,Lft Pupil Diameter,Rt X Pos,Rt Y Pos,...,Head Rot (X),Head Rot (Y),Head Rot (Z),Head Pos (X),Head Pos (Y),Head Pos (Z),L Eyelid Valid,L Eyelid Opening,R Eyelid Valid,R Eyelid Opening
0,39097,40409,71789578202,1581624104695800,71789417767,0.0000,0.0000,0.000000,0.0000,0.0000,...,0,0,0,0.000000,0.000000,0.000000,0,0,0,0
1,39098,40410,71789595159,1581624104712760,71789433958,0.0000,0.0000,0.000000,0.0000,0.0000,...,0,0,0,0.000000,0.000000,0.000000,0,0,0,0
2,39099,40411,71789622078,1581624104739670,71789451202,0.0000,0.0000,0.000000,0.0000,0.0000,...,0,0,0,0.000000,0.000000,0.000000,0,0,0,0
3,39100,40412,71789625068,1581624104742660,71789466661,0.0000,0.0000,0.000000,0.0000,0.0000,...,0,0,0,0.000000,0.000000,0.000000,0,0,0,0
4,39101,40413,71789646185,1581624104763780,71789482596,0.0000,0.0000,0.000000,0.0000,0.0000,...,0,0,0,0.000000,0.000000,0.000000,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13216,52313,53625,72004012114,1581624319129710,72003841288,1007.5436,406.4036,3.268553,1007.5436,406.4036,...,0,0,0,-0.022282,0.272468,0.507865,0,0,0,0
13217,52314,53626,72004024075,1581624319141670,72003858295,1012.7636,405.9277,3.370696,1012.7636,405.9277,...,0,0,0,-0.021631,0.274314,0.513835,0,0,0,0
13218,52315,53627,72004036040,1581624319153640,72003873570,1018.0250,405.0260,3.343998,1018.0250,405.0260,...,0,0,0,-0.021328,0.273554,0.511405,0,0,0,0
13219,52316,53628,72004055981,1581624319173580,72003889708,1023.9369,403.5870,3.367016,1023.9369,403.5870,...,0,0,0,-0.020667,0.274904,0.515763,0,0,0,0


In [201]:
df1 = data.loc[:,['Sensor Time','Lft X Pos', 'Lft Y Pos', 'Lft Pupil Diameter', 'Rt X Pos', 'Rt Y Pos',]]

In [202]:
df1['gaze_x'] = (df1['Lft X Pos'] + df1['Rt X Pos'])/2
df1['gaze_y'] = (df1['Lft Y Pos'] + df1['Rt Y Pos'])/2
df1['time_convert']=df1['Sensor Time']/1000000

In [203]:
#function to remove missing data which is the blink
def remove_missing_data(df,missing = 0):
    mx=np.array(np.array(df.gaze_x)==missing, dtype =int)
    my=np.array(np.array(df.gaze_y)==missing, dtype =int)
    eye_x = df.gaze_x[mx != 1]
    eye_y = df.gaze_x[my != 1]
    time = df['time_convert'][mx != 1]
    return eye_x,eye_y,time

In [204]:
#fixation duration function
def fixation_detection(df, mindur=15):
    x, y, time = remove_missing_data(df, missing=0)
    Sfix = []
    Efix = []
    fixationstart = False
    for i in range(1, len(x)):
        try:
            gap = abs(int((time[i - 1] - time[i]) * 1000))
            if gap <= mindur and not fixationstart:
                fixationstart = True
                Sfix.append(time[i])
            if gap > mindur and fixationstart:
                fixationstart = False
                #the time data type was changed to float from integer to find the exact values
                Efix.append([Sfix[-1], time[i - 1], float(time[i - 1] - Sfix[-1]), x[i], y[i]])
        except KeyError:
            pass
    return Efix

In [188]:
fixation_data = fixation_detection(df1,15)

In [205]:
fixation_data

[[71792.078425, 71792.15959, 0.08116499999596272, 122.51395, 85.75965],
 [71792.224273, 71792.224273, 0.0, 125.74705, 80.16295],
 [71792.581302, 71792.581302, 0.0, 635.9924, 601.664],
 [71792.613691, 71792.613691, 0.0, 634.576, 603.041],
 [71792.662315, 71792.662315, 0.0, 633.1673, 606.5271],
 [71792.710845, 71792.710845, 0.0, 634.4659, 612.8474],
 [71792.75946, 71792.75946, 0.0, 633.888, 613.6031],
 [71792.808526, 71792.824505, 0.015979000003426336, 631.6988, 616.3686],
 [71792.873089, 71792.873089, 0.0, 628.0553, 614.2079],
 [71792.93825, 71792.93825, 0.0, 626.2133, 618.3917],
 [71792.970905, 71792.970905, 0.0, 627.0265, 617.0824],
 [71793.00503, 71793.019652, 0.014622000002418645, 624.9641, 613.1462],
 [71793.068162, 71793.068162, 0.0, 620.9343, 611.2336],
 [71793.116814, 71793.116814, 0.0, 620.2367, 613.7307],
 [71793.165502, 71793.165502, 0.0, 617.4757, 615.3304],
 [71793.214337, 71793.214337, 0.0, 619.7688, 615.3065],
 [71793.263024, 71793.263024, 0.0, 621.0025, 618.5651],
 [7179

In [190]:
#from the faxation data above we isolated the duration of the fixation and stored it in a variable 

fixation_druation = [sublist[2] for sublist in fixation_data]

In [194]:
fixation_druation

[0.08116499999596272,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.015979000003426336,
 0.0,
 0.0,
 0.0,
 0.014622000002418645,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.015950999993947335,
 0.014595999993616715,
 0.0,
 0.0,
 0.01594499999191612,
 0.0,
 0.0,
 0.0,
 0.015834999998332933,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.015633999995770864,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.015899999998509884,
 0.0,
 0.0,
 0.015463999996427447,
 0.0,
 0.0,
 0.0,
 0.0,
 0.01578499999595806,
 0.015795999992405996,
 0.0,
 0.0,
 0.0,
 0.0,
 0.015533999991021119,
 0.015637999997125007,
 0.0,
 0.0,
 0.0,
 0.0,
 0.01564199999847915,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.015897000004770234,
 0.01589300000341609,
 0.1457419999933336,
 0.015683999998145737,
 0.0,
 0.01587699999799952,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.01565100000880193,
 0.0,
 0.0,
 0.0,
 0.015820000000530854,
 0.0,
 0.0,
 0.0,
 0.015839999992749654,
 0.01549600000726059,
 0

## Function to find fixation duration and total fixation count

In [213]:
#function to show the average fixation duration and total fixation count

def find_avgdur_totalcount(fixation_data):
    print("The average fixation duration is:",(sum(fixation_data) / len(fixation_data)))
    print("The total fixation count is:",len(fixation_data))

In [212]:
find_avgdur_totalcount(fixation_druation)

The average fixation duration is: 0.01535582226216374
The total fixation count is: 2228
